In [1]:
"""
Linear Regression :

"""

'\nLinear Regression :\n\n'

In [2]:
import warnings

warnings.filterwarnings('ignore')

import pandas as pd 
import numpy as np

from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

from testPipeline import *

In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale

In [4]:
# filepath = os.path.abspath(os.path.join('..', 'data', 'UK_house_data.csv'))
# print('File path: {}'.format(filepath))

In [5]:
train_file=r'../data/loan_data_train.csv'
test_file=r'../data/loan_data_test.csv'

ld_train=pd.read_csv(train_file)
ld_test=pd.read_csv(test_file)      

In [6]:
ld_train.head(10)

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,79542.0,25000,25000,18.49%,60 months,debt_consolidation,27.56%,VA,MORTGAGE,8606.56,720-724,11,15210,3.0,5 years
1,75473.0,19750,19750,17.27%,60 months,debt_consolidation,13.39%,NY,MORTGAGE,6737.50,710-714,14,19070,3.0,4 years
2,67265.0,2100,2100,14.33%,36 months,major_purchase,3.50%,LA,OWN,1000.00,690-694,13,893,1.0,< 1 year
3,80167.0,28000,28000,16.29%,36 months,credit_card,19.62%,NV,MORTGAGE,7083.33,710-714,12,38194,1.0,10+ years
4,17240.0,24250,17431.82,12.23%,60 months,credit_card,23.79%,OH,MORTGAGE,5833.33,730-734,6,31061,2.0,10+ years
5,32737.0,5400,5375,8.90%,36 months,small_business,6.27%,OH,RENT,3000.00,785-789,2,90,2.0,NaN
6,71685.0,16000,16000,22.47%,60 months,credit_card,11.27%,CA,RENT,4791.67,665-669,5,8474,0.0,2 years
7,38028.0,6400,6400,7.51%,36 months,major_purchase,9.58%,IL,RENT,6400.00,735-739,11,12306,1.0,< 1 year
8,32119.0,24000,23772.15,12.99%,60 months,home_improvement,15.54%,NC,MORTGAGE,8333.33,740-744,24,56984,0.0,NaN
9,84563.0,8000,8000,7.62%,36 months,other,0%,NC,MORTGAGE,3500.00,765-769,7,0,1.0,NaN


In [7]:
ld_test.head(10)

,ID,Amount.Requested,Amount.Funded.By.Investors,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,20093,5000,5000,60 months,moving,12.59%,NY,RENT,4416.67,690-694,13,7686,0,< 1 year
1,62445,18000,18000,60 months,debt_consolidation,4.93%,CA,RENT,5258.50,710-714,6,11596,0,10+ years
2,65248,7200,7200,60 months,debt_consolidation,25.16%,LA,MORTGAGE,3750.00,750-754,13,7283,0,6 years
3,81822,7200,7200,36 months,debt_consolidation,17.27%,NY,MORTGAGE,3416.67,790-794,14,4838,0,10+ years
4,57923,22000,22000,60 months,debt_consolidation,18.28%,MI,MORTGAGE,6083.33,720-724,9,20181,0,8 years
5,33777,6000,.,36 months,home_improvement,2.52%,NJ,MORTGAGE,4800.00,750-754,.,1442,0,10+ years
6,22573,9800,9800,60 months,credit_card,12.89%,CA,MORTGAGE,2708.33,690-694,11,8111,0,3 years
7,32044,6000,6000,36 months,major_purchase,8.81%,NY,OWN,2462.75,715-719,11,8644,0,2 years
8,8127,3000,3000,36 months,moving,10.05%,MA,RENT,9166.67,665-669,15,22698,2,6 years
9,6067,17500,17375,36 months,debt_consolidation,21.63%,CO,RENT,3666.67,715-719,7,20010,0,6 years


In [8]:
ld_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              2199 non-null   float64
 1   Amount.Requested                2199 non-null   object 
 2   Amount.Funded.By.Investors      2199 non-null   object 
 3   Interest.Rate                   2200 non-null   object 
 4   Loan.Length                     2199 non-null   object 
 5   Loan.Purpose                    2199 non-null   object 
 6   Debt.To.Income.Ratio            2199 non-null   object 
 7   State                           2199 non-null   object 
 8   Home.Ownership                  2199 non-null   object 
 9   Monthly.Income                  2197 non-null   float64
 10  FICO.Range                      2200 non-null   object 
 11  Open.CREDIT.Lines               2196 non-null   object 
 12  Revolving.CREDIT.Balance        21

In [9]:
ld_train.sample(10)

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
1589,49074.0,12375,12375,13.67%,36 months,debt_consolidation,22.42%,KS,MORTGAGE,2083.33,680-684,8,12226,0.0,3 years
1486,55180.0,9175,9175,13.67%,36 months,credit_card,19.43%,IL,MORTGAGE,7083.33,675-679,8,25639,0.0,8 years
920,97524.0,12000,12000,6.03%,36 months,debt_consolidation,18.75%,SC,MORTGAGE,13000.00,725-729,22,20197,0.0,5 years
835,82780.0,18000,18000,11.14%,36 months,debt_consolidation,11.81%,FL,RENT,5100.00,715-719,8,17139,0.0,1 year
96,27520.0,5500,5500,15.99%,60 months,debt_consolidation,17.95%,RI,RENT,4250.00,700-704,16,13256,0.0,10+ years
754,23644.0,2700,2700,5.79%,36 months,debt_consolidation,19.81%,OH,MORTGAGE,3958.33,750-754,14,13159,1.0,8 years
2022,30986.0,13800,13800,17.49%,60 months,medical,1.68%,NY,RENT,4166.67,710-714,3,2766,0.0,8 years
379,60875.0,10500,10500,12.12%,36 months,major_purchase,5.42%,NY,RENT,5000.00,700-704,4,502,0.0,2 years
1449,10264.0,15200,6800,15.58%,60 months,home_improvement,19.22%,DC,MORTGAGE,12083.33,690-694,20,19712,2.0,2 years
747,8194.0,6525,6500,16.07%,36 months,other,10.11%,MD,RENT,3166.67,700-704,3,2494,1.0,1 year


In [10]:
# drop columns : Amount.Funded.By.Investors , ID, Interest.Rate

#1 Amount requested : convert it to numeric

#2 Loan Length : create dummies with frequency cutoff 20

#3 Loan.Purpose : dummies with freq cutoff

#4  Debt.To.Income.Ratio : remove % and then convert to numeric

#5  State: dummies with frequency cutoff

#6 Home.Ownership : dummies with frequency cutoff

#7 Monthly Income : as is

#8 FICO.Range : break a-b , in to a ,b , convert them to numeric 
# then create new column fico=0.5*(a+b) and then drop the original FICO.Range

#9 Open Credit Lines : convert to numeric

#10 Revolving Credit balance : convert it to numeric 

#11 inquiries in the last 6 months : as is

#12 Employment.Length : create dummies with frequency cutoff

In [11]:
"""
Creating sub pipelines 
"""

'\nCreating sub pipelines \n'

In [12]:
p1=pdPipeline([
    ('var_select',VarSelector(['Amount.Requested','Open.CREDIT.Lines','Revolving.CREDIT.Balance'])),
    ('convert_to_numeric',convert_to_numeric()),
    ('missing_trt',DataFrameImputer())
])

p2=pdPipeline([
    ('var_select',VarSelector(['Debt.To.Income.Ratio'])),
    ('string_clean',string_clean(replace_it='%',replace_with='')),
    ('convert_to_numeric',convert_to_numeric()),
    ('missing_trt',DataFrameImputer())
])

p3=pdPipeline([
    ('var_select',VarSelector(['Loan.Length', 'Loan.Purpose','State','Home.Ownership','Employment.Length'])),
    ('missing_trt',DataFrameImputer()),
    ('create_dummies',get_dummies_Pipe(20))
])

p4=pdPipeline([
    ('var_select',VarSelector(['Monthly.Income','Inquiries.in.the.Last.6.Months'])),
    ('missing_trt',DataFrameImputer())
])

p5=pdPipeline([
    ('var_select',VarSelector(['FICO.Range'])),
    ('custom_fico',custom_fico()),
    ('missing_trt',DataFrameImputer())
])



In [13]:
"""
Feature Union of the sub pipelines
"""

data_pipe=FeatureUnion([
    ('obj_to_num',p1),
    ('dtir',p2),
    ('obj_to_dum',p3),
    ('num',p4),
    ('fico',p5)
])

In [14]:
"""
Transforming the  training data : 
"""

x_train=pd.DataFrame(data=data_pipe.fit_transform(ld_train),
                     columns=data_pipe.get_feature_names())

In [15]:
x_train.shape

(2200, 60)

In [16]:
x_train.sample(10)


,obj_to_num__Amount.Requested,obj_to_num__Open.CREDIT.Lines,obj_to_num__Revolving.CREDIT.Balance,dtir__Debt.To.Income.Ratio,obj_to_dum__Loan.Length_36 months,obj_to_dum__Loan.Length_60 months,obj_to_dum__Loan.Purpose_debt_consolidation,obj_to_dum__Loan.Purpose_credit_card,obj_to_dum__Loan.Purpose_other,obj_to_dum__Loan.Purpose_home_improvement,...,obj_to_dum__Employment.Length_4 years,obj_to_dum__Employment.Length_1 year,obj_to_dum__Employment.Length_6 years,obj_to_dum__Employment.Length_7 years,obj_to_dum__Employment.Length_8 years,obj_to_dum__Employment.Length_missing,obj_to_dum__Employment.Length_9 years,num__Monthly.Income,num__Inquiries.in.the.Last.6.Months,fico__fico
28,24000.0,8.0,11058.0,14.22,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6833.33,0.0,762.0
1352,4800.0,8.0,5093.0,5.31,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3500.00,0.0,692.0
617,12000.0,10.0,32028.0,19.41,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,8750.00,1.0,712.0
489,10000.0,11.0,9261.0,12.80,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6666.67,0.0,732.0
797,10000.0,8.0,3822.0,17.80,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2500.00,2.0,722.0
554,10000.0,4.0,0.0,7.77,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3166.67,0.0,737.0
246,8800.0,5.0,6055.0,4.89,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5479.00,0.0,757.0
342,10000.0,11.0,14426.0,19.01,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7500.00,1.0,682.0
240,12000.0,8.0,9866.0,13.18,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5833.33,0.0,727.0
361,18200.0,13.0,16184.0,7.95,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4750.00,2.0,722.0


In [17]:
x_test=pd.DataFrame(data=data_pipe.transform(ld_test),
                     columns=data_pipe.get_feature_names())

In [18]:
x_test.shape

(300, 60)

In [19]:
x_test.head()

,obj_to_num__Amount.Requested,obj_to_num__Open.CREDIT.Lines,obj_to_num__Revolving.CREDIT.Balance,dtir__Debt.To.Income.Ratio,obj_to_dum__Loan.Length_36 months,obj_to_dum__Loan.Length_60 months,obj_to_dum__Loan.Purpose_debt_consolidation,obj_to_dum__Loan.Purpose_credit_card,obj_to_dum__Loan.Purpose_other,obj_to_dum__Loan.Purpose_home_improvement,...,obj_to_dum__Employment.Length_4 years,obj_to_dum__Employment.Length_1 year,obj_to_dum__Employment.Length_6 years,obj_to_dum__Employment.Length_7 years,obj_to_dum__Employment.Length_8 years,obj_to_dum__Employment.Length_missing,obj_to_dum__Employment.Length_9 years,num__Monthly.Income,num__Inquiries.in.the.Last.6.Months,fico__fico
0,5000.0,13.0,7686.0,12.59,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4416.67,0.0,692.0
1,18000.0,6.0,11596.0,4.93,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5258.50,0.0,712.0
2,7200.0,13.0,7283.0,25.16,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3750.00,0.0,752.0
3,7200.0,14.0,4838.0,17.27,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3416.67,0.0,792.0
4,22000.0,9.0,20181.0,18.28,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6083.33,0.0,722.0


In [20]:
y_train=ld_train['Interest.Rate'].str.replace('%','').astype(float)

In [21]:
y_train

0       18.49
1       17.27
2       14.33
3       16.29
4       12.23
        ...  
2195    23.28
2196    14.33
2197    15.31
2198    20.99
2199    10.16
Name: Interest.Rate, Length: 2200, dtype: float64

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [23]:
lm=LinearRegression()

In [24]:
cv_mae=-cross_val_score(lm,
                        x_train,y_train,
                        cv=10,
                        scoring='neg_mean_absolute_error')

In [25]:
cv_mae

array([1.74273801, 1.76502624, 1.77011354, 1.65406589, 1.43720296,
       1.63540955, 1.44255408, 1.58043799, 1.52220714, 1.65823135])

In [26]:
cv_mae.mean()

1.6207986760157767

In [27]:
cv_mae.std()

0.11726675023099994

In [28]:
"""
Fit x_train , y_train
"""
lm.fit(x_train,y_train)

LinearRegression()

In [29]:
lm.intercept_

73.16432302814546

In [30]:
lm.coef_

array([ 1.60913878e-04, -3.72164153e-02, -3.05844966e-06,  1.88427404e-04,
        1.30627284e+00,  4.47755390e+00, -7.84079703e-01, -8.91888331e-01,
        3.20936459e-02, -6.95275775e-01, -4.22744797e-01, -2.64224120e-01,
       -5.41274155e-01, -9.69555836e-01, -5.98574359e-01,  7.21470875e-01,
       -1.99766458e-01, -1.88645568e-01, -7.45527597e-02,  4.73923022e-01,
       -4.68212871e-01, -5.71694597e-01, -1.79076702e-01, -3.78950003e-01,
       -1.57191036e-01, -1.08257898e-01, -4.96139885e-01, -1.99177682e-01,
       -7.78964606e-02,  4.10456351e-02, -2.67193758e-01,  4.08575782e-01,
        3.80179542e-01, -9.81649463e-02, -6.21920499e-02, -1.77735228e-01,
       -3.50604056e-01, -2.33745724e-01, -3.92591988e-02,  1.10048502e-02,
        1.87183988e-02,  1.85782841e-01, -2.39881640e+00, -2.17035525e+00,
       -2.06526514e+00,  5.26184321e-01,  3.52218686e-01,  2.87554381e-01,
        1.51007910e-01,  5.83377246e-01,  4.22222352e-01,  2.41210867e-01,
        3.88741640e-01,  

In [31]:
list(zip(x_train.columns,lm.coef_))

[('obj_to_num__Amount.Requested', 0.0001609138780966344),
 ('obj_to_num__Open.CREDIT.Lines', -0.03721641529308709),
 ('obj_to_num__Revolving.CREDIT.Balance', -3.058449663781562e-06),
 ('dtir__Debt.To.Income.Ratio', 0.00018842740385924261),
 ('obj_to_dum__Loan.Length_36 months', 1.3062728364927083),
 ('obj_to_dum__Loan.Length_60 months', 4.4775539032857345),
 ('obj_to_dum__Loan.Purpose_debt_consolidation', -0.7840797027391835),
 ('obj_to_dum__Loan.Purpose_credit_card', -0.8918883314498496),
 ('obj_to_dum__Loan.Purpose_other', 0.03209364593789732),
 ('obj_to_dum__Loan.Purpose_home_improvement', -0.6952757746460749),
 ('obj_to_dum__Loan.Purpose_major_purchase', -0.42274479675685656),
 ('obj_to_dum__Loan.Purpose_small_business', -0.26422412044756144),
 ('obj_to_dum__Loan.Purpose_car', -0.5412741545798316),
 ('obj_to_dum__Loan.Purpose_wedding', -0.9695558362067362),
 ('obj_to_dum__Loan.Purpose_medical', -0.5985743591782668),
 ('obj_to_dum__Loan.Purpose_moving', 0.7214708748508568),
 ('obj_t

In [32]:
"""
Now , lets Predict using x_test
"""

'\nNow , lets Predict using x_test\n'

In [33]:
test_pred=lm.predict(x_test)

In [34]:
pd.DataFrame(test_pred).to_csv("Predicted_InterestRates.csv",index=False)


In [35]:
# y_test=ld_test['Interest.Rate'].str.replace('%','').astype(float)

x_test.columns

Index(['obj_to_num__Amount.Requested', 'obj_to_num__Open.CREDIT.Lines',
       'obj_to_num__Revolving.CREDIT.Balance', 'dtir__Debt.To.Income.Ratio',
       'obj_to_dum__Loan.Length_36 months',
       'obj_to_dum__Loan.Length_60 months',
       'obj_to_dum__Loan.Purpose_debt_consolidation',
       'obj_to_dum__Loan.Purpose_credit_card',
       'obj_to_dum__Loan.Purpose_other',
       'obj_to_dum__Loan.Purpose_home_improvement',
       'obj_to_dum__Loan.Purpose_major_purchase',
       'obj_to_dum__Loan.Purpose_small_business',
       'obj_to_dum__Loan.Purpose_car', 'obj_to_dum__Loan.Purpose_wedding',
       'obj_to_dum__Loan.Purpose_medical', 'obj_to_dum__Loan.Purpose_moving',
       'obj_to_dum__State_CA', 'obj_to_dum__State_NY', 'obj_to_dum__State_FL',
       'obj_to_dum__State_TX', 'obj_to_dum__State_PA', 'obj_to_dum__State_IL',
       'obj_to_dum__State_GA', 'obj_to_dum__State_NJ', 'obj_to_dum__State_VA',
       'obj_to_dum__State_MA', 'obj_to_dum__State_NC', 'obj_to_dum__State_OH',


In [36]:
"""
now lets calculate the MSE RMSE : 

"""

'\nnow lets calculate the MSE RMSE : \n\n'

In [37]:
mse = mean_squared_error(ytest, test_pred)

NameError: name 'ytest' is not defined